In [1]:
#with misepcificed moments everything depends on the limiting behavior of hte weighting matrix. it may not coverge?
#looks like the issue is that GMM gives weight to the bad assumptions?

In [49]:
import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import GMM
import numpy as np

from scipy.optimize import minimize

In [50]:
def gen_data(nobs=1000, a=.5,k=1):
    
    #error terms
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    u = np.random.normal(loc=0.0, scale=5.0, size=nobs)
    
    z = np.random.normal(scale=1., size=(nobs,k))
    x = a*z.sum(axis=1) + u
    y = 1 + 1*x + e
    return y,sm.add_constant(x),z.reshape(nobs,k),nobs

#the simplest example is mu = avg(x) and mu = avg(y)? only one can be true?
#could also have an example where z is endogenous and x is not, or z is more endogenous than x?

In [51]:
def compute_moments_iv(params,y,x,z):   
        iv = []
        ols = []
        
        e = y- x.dot(params)
        
        for col in range(z.shape[1]):
            iv.append(z[:,col]*e)
            
        #for col in range(x.shape[1]):
        #    ols.append(x[:,col]*e)
            
        g = np.concatenate(([e],iv),axis=0)
        g = g.T
        return g

def gmm_obj_iv(params,y,x,z):
    moms = compute_moments_iv(params,y,x,z).mean(axis=0)
    moms = (moms**2).mean()
    return moms


def compute_moments_ols(params,y,x,z):   
        iv = []
        ols = []
        
        e = y- x.dot(params)
        
        for col in range(x.shape[1]):
            ols.append(x[:,col]*e)
            
        g = np.array(ols)
        g = g.T
        return g

def gmm_obj_ols(params,y,x,z):
    moms = compute_moments_ols(params,y,x,z).mean(axis=0)
    moms = (moms**2).mean()
    return moms


def compute_moments(params,y,x,z):   
        iv = []
        ols = []
        
        e = y- x.dot(params)
        
        for col in range(z.shape[1]):
            iv.append(z[:,col]*e)
            
        for col in range(x.shape[1]):
            ols.append(x[:,col]*e)
            
        g = np.concatenate((ols,iv),axis=0)
        g = g.T
        return g

def gmm_obj(params,y,x,z):
    moms = compute_moments(params,y,x,z).mean(axis=0)
    moms = (moms**2).mean()
    return moms



y,x,z,nobs = gen_data()

res = minimize( gmm_obj, [1,1],  args=(y,x,z))
print(res.x)
print(np.cov(compute_moments(res.x,y,x,z),rowvar=False))
print('--')

res = minimize( gmm_obj_ols, [1,1],  args=(y,x,z))
print(res.x)
print(np.cov(compute_moments_ols(res.x,y,x,z),rowvar=False))
print('--')

res = minimize( gmm_obj_iv, [1,1],  args=(y,x,z))
print(res.x)
print(np.cov(compute_moments_iv(res.x,y,x,z),rowvar=False))
print('--')

#looks like the issue is that GMM gives weight to the bad assumptions?

[0.98618966 0.9966999 ]
[[ 0.89563198  0.24017535  0.04311353]
 [ 0.24017535 22.00288576  0.57311959]
 [ 0.04311353  0.57311959  0.91833793]]
--
[0.98670461 0.99675738]
[[ 0.8956319   0.24010922]
 [ 0.24010922 22.00239712]]
--
[0.97622125 0.9024941 ]
[[1.12073615 0.03069103]
 [0.03069103 1.17137641]]
--


In [26]:
class OLS_IV(GMM):
    
    def __init__(self, *args, **kwds):
        # set appropriate counts for moment conditions and parameters
        super(OLS_IV, self).__init__(*args, **kwds)
        self.nmoms = self.momcond(self.fitstart()).shape[1]

    def fitstart(self):
        """Create array of zeros"""
        return np.zeros(self.exog.shape[1])
    
    
    def start_weights(self, inv=True):
        """Create identity matrix for starting weights"""
        
        return np.eye(self.nmoms)
    
    
    def momcond(self, params):
        #unwrap stuff
        y,x,z = self.endog, self.exog, self.instrument

        return compute_moments(params,y,x,z)
    


model = OLS_IV(y , x, z)
model.fit(maxiter=0).summary()

Optimization terminated successfully.
         Current function value: 0.000430
         Iterations: 4
         Function evaluations: 6
         Gradient evaluations: 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                                OLS_IV Results                                
==============================================================================
Dep. Variable:                      y   Hansen J:                       0.4300
Model:                         OLS_IV   Prob (Hansen J):                 0.512
Method:                           GMM                                         
Date:                Wed, 30 Mar 2022                                         
Time:                        11:33:27                                         
No. Observations:                1000                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0071      0.032     31.680      0.000       0.945       1.069
x1             1.0029      0.029     35.156      0.000       0.947       1.059
==============================================================================
"""